In [82]:
import h3
import pandas as pd
import numpy as np

# Creating Main Dataframe

## Banan Slug Observation Data

In [83]:
observation_df = pd.read_csv('banana_slug_observations.csv', low_memory=False)

In [84]:
observation_df.head(4).T

,0,1,2,3
id,424,1831,4166,5029
uuid,1126eca5-7b8d-4486-b30e-3ba3c19adf2f,d1bc6ded-fe4c-4986-82bc-8bece59bc467,fd46b967-429c-4527-aae4-1ea2ab193eac,606e2e62-4fae-4080-b828-cb7a7020f987
observed_on_string,05/24/2008,"March 07, 2009 10:45",2010-01-10,"December 15, 2007"
observed_on,2008-05-24,2009-03-07,2010-01-10,2007-12-15
time_observed_at,NaN,2009-03-07 18:45:00 UTC,NaN,NaN
time_zone,Pacific Time (US & Canada),Pacific Time (US & Canada),Pacific Time (US & Canada),Pacific Time (US & Canada)
user_id,49,32,307,354
updated_at,2016-12-27 22:44:54 UTC,2024-04-07 05:37:20 UTC,2024-05-21 14:50:16 UTC,2020-03-25 15:04:03 UTC
quality_grade,research,research,research,research
license,NaN,NaN,NaN,CC-BY


---

## Converting to h3 geospatial grid

In [85]:
observation_df['h3_level_9'] = observation_df.apply(
    lambda row: h3.latlng_to_cell(row['latitude'], row['longitude'], 9), 
    axis=1
)

In [87]:
df = pd.DataFrame(observation_df['observed_on'])

weatherbit_df = pdDataFrame(
    ['Custom ID
    ['latitude'],
    ['longitude']

In [88]:
df['observed_on'] = pd.to_datetime(df['observed_on'])
df['latitude'] = observation_df['latitude']
df['longitude'] = observation_df['longitude']

In [89]:
df = df.set_index('observed_on')

In [90]:
df.to_csv('banana_slug_lat_long.csv', index=True)

In [91]:
df

,latitude,longitude
observed_on,,
2008-05-24,40.307499,-123.908997
2009-03-07,37.894199,-122.237999
2010-01-10,37.309986,-122.181923
2007-12-15,37.842258,-122.555923
2006-04-15,37.573944,-122.466537
...,...,...
2024-11-10,44.645570,-123.411087
2025-03-12,48.521267,-123.549780
2006-04-01,48.470234,-122.628522


---

## Weather Data

In [142]:
test = pd.DataFrame({
    '5': observation_df.apply(
        lambda row: h3.latlng_to_cell(row['latitude'], row['longitude'], 5), 
        axis=1
    ),
    '9': observation_df['h3_level_9']
})

In [146]:
test['9'].nunique()

21501

In [104]:
import requests
import json
from datetime import datetime

In [110]:
weatherbit_url = 'https://api.weatherbit.io/v2.0/history/daily'

In [111]:
lat = h3.cell_to_latlng(observation_df['h3_level_5'][0])[0]

In [112]:
lat

40.29218733169173

In [113]:
lon = h3.cell_to_latlng(observation_df['h3_level_5'][0])[1]

In [114]:
lon

-123.90223856220408

In [131]:
params = {
    "lat":lat,
    "lon":lon,
    'start_date': '2008-06-24',
    'end_date': '2008-06-24',
    'key': API
}

In [132]:
#response = requests.get(weatherbit_url, params=params)

In [133]:
if response.status_code != 200:
    print(f"Error: {response.text}")

Error: {"error":"end_date is equal to start_date. end_date should be after start_date."}



In [135]:
parsed_json = json.loads(json_str)
# Extract just the data array
data_array = parsed_json.get('data', [])
print("\nData extracted with json directly:")
print(data_array)



Data extracted with json directly:
[{'rh': 70.2, 'wind_spd': 3.8, 'slp': 1022, 'max_wind_spd': 6.7, 'max_wind_dir': 220, 'max_wind_spd_ts': 1483232400, 'wind_gust_spd': 12.7, 'min_temp_ts': 1483272000, 'max_temp_ts': 1483308000, 'dewpt': 1.8, 'snow': 0, 'snow_depth': 1.0, 'precip': 10.5, 'precip_gpm': 13.5, 'wind_dir': 189, 'max_dhi': 736.3, 'dhi': 88, 'max_temp': 10, 'pres': 1006.4, 'max_uv': 5, 't_dhi': 2023.6, 'datetime': '2025-04-15', 'temp': 7.86, 'min_temp': 5, 'clouds': 43, 'ts': 1483228800, 'revision_status': 'final'}]


In [136]:
test = pd.DataFrame(data_array)

In [137]:
test

,rh,wind_spd,slp,max_wind_spd,max_wind_dir,max_wind_spd_ts,wind_gust_spd,min_temp_ts,max_temp_ts,dewpt,...,max_temp,pres,max_uv,t_dhi,datetime,temp,min_temp,clouds,ts,revision_status
0,70.2,3.8,1022,6.7,220,1483232400,12.7,1483272000,1483308000,1.8,...,10,1006.4,5,2023.6,2025-04-15,7.86,5,43,1483228800,final
